In [1]:
import pandas as pd
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pickle
import sqlite3
from database_connector import *
import warnings
warnings.filterwarnings('ignore')

In [2]:
course_tags = pd.read_csv("data/course_tags.csv")
user_assesment = pd.read_csv("data/user_assessment_scores.csv")
user_course_view= pd.read_csv("data/user_course_views.csv")
user_interests = pd.read_csv("data/user_interests.csv")

In [3]:
print("Course tags dim: ", course_tags.shape)
print("User assessment dim: ", user_assesment.shape)
print("User course view dim: ", user_course_view.shape)
print("User interests dim: ", user_interests.shape)

Course tags dim:  (11337, 2)
User assessment dim:  (6571, 4)
User course view dim:  (249238, 6)
User interests dim:  (297526, 3)


In [4]:
print(course_tags.head())
print('\n')
print(user_assesment.head())

                                        course_id       course_tags
0  12-principles-animation-toon-boom-harmony-1475      2d-animation
1              2d-racing-game-series-unity-5-1312       game-design
2             2d-racing-games-unity-volume-2-1286          game-art
3             2d-racing-games-unity-volume-2-1286  digital-painting
4             2d-racing-games-unity-volume-2-1286     image-editing


   user_handle assessment_tag user_assessment_date  user_assessment_score
0         7487     angular-js  2017-08-11 19:03:38                    134
1         7487            css  2017-08-11 20:09:56                     38
2         7487          html5  2017-07-31 18:59:37                     84
3         7487           java  2017-07-31 18:49:27                    149
4         7487     javascript  2017-07-31 19:05:03                     92


In [5]:
print(user_course_view.head())
print('\n')
print(user_interests.head())

   user_handle   view_date                                       course_id  \
0            1  2017-06-27         cpt-sp2010-web-designers-branding-intro   
1            1  2017-06-28         cpt-sp2010-web-designers-branding-intro   
2            1  2017-06-28                    cpt-sp2010-web-designers-css   
3            1  2017-07-27                    cpt-sp2010-web-designers-css   
4            1  2017-09-12  aws-certified-solutions-architect-professional   

   author_handle         level  view_time_seconds  
0            875      Beginner               3786  
1            875      Beginner               1098  
2            875  Intermediate               4406  
3            875  Intermediate                553  
4            281      Advanced                102  


   user_handle      interest_tag        date_followed
0            1   mvc-scaffolding  2017-06-27 16:26:52
1            1              mvc2  2017-06-27 16:26:52
2            1  mvc-html-helpers  2017-06-27 16:26:52
3

In [6]:
print(user_assesment["user_handle"].nunique())
print(course_tags["course_id"].nunique())
print(user_assesment["assessment_tag"].nunique())
print(course_tags["course_tags"].nunique())
print(user_course_view["user_handle"].nunique())
print(user_course_view["author_handle"].nunique())

3114
5942
54
998
8760
1412


## Merging two generic data frames that reflect user engagement    
   - This will help to understand similar user interests based on their activity i.e. what courses users view, level of difficulty etc.

In [7]:
df_user_attr = user_course_view.join(user_interests, on='user_handle', how='left', lsuffix='_x', rsuffix='_y')
df_user_attr.head()

,user_handle_x,view_date,course_id,author_handle,level,view_time_seconds,user_handle_y,interest_tag,date_followed
0,1,2017-06-27,cpt-sp2010-web-designers-branding-intro,875,Beginner,3786,1,mvc2,2017-06-27 16:26:52
1,1,2017-06-28,cpt-sp2010-web-designers-branding-intro,875,Beginner,1098,1,mvc2,2017-06-27 16:26:52
2,1,2017-06-28,cpt-sp2010-web-designers-css,875,Intermediate,4406,1,mvc2,2017-06-27 16:26:52
3,1,2017-07-27,cpt-sp2010-web-designers-css,875,Intermediate,553,1,mvc2,2017-06-27 16:26:52
4,1,2017-09-12,aws-certified-solutions-architect-professional,281,Advanced,102,1,mvc2,2017-06-27 16:26:52


In [8]:
### Mapping the course tags with the course ids for better understanding

m = dict(zip(course_tags["course_id"], course_tags["course_tags"]))
df_user_attr["course_tags"] = df_user_attr["course_id"].map(lambda x: m.get(x, "not available"))
df_user_attr.drop(["view_date", "course_id", "date_followed"], axis=1, inplace=True)
df_user_attr.head()

,user_handle_x,author_handle,level,view_time_seconds,user_handle_y,interest_tag,course_tags
0,1,875,Beginner,3786,1,mvc2,sharepoint
1,1,875,Beginner,1098,1,mvc2,sharepoint
2,1,875,Intermediate,4406,1,mvc2,sharepoint
3,1,875,Intermediate,553,1,mvc2,sharepoint
4,1,281,Advanced,102,1,mvc2,cloud-services


In [9]:
### Appending user assessment scores of user that have those scores

p = dict(zip(user_assesment["user_handle"], user_assesment["assessment_tag"]))
n = dict(zip(user_assesment["user_handle"], user_assesment["user_assessment_score"]))
df_user_attr["user_assessment_score"] = df_user_attr["user_handle_x"].map(lambda x: m.get(x,0))
df_user_attr["assessment_tag"] = df_user_attr["user_handle_x"].map(lambda x: p.get(x,"not available"))
df_user_attr.head()

,user_handle_x,author_handle,level,view_time_seconds,user_handle_y,interest_tag,course_tags,user_assessment_score,assessment_tag
0,1,875,Beginner,3786,1,mvc2,sharepoint,0,not available
1,1,875,Beginner,1098,1,mvc2,sharepoint,0,not available
2,1,875,Intermediate,4406,1,mvc2,sharepoint,0,not available
3,1,875,Intermediate,553,1,mvc2,sharepoint,0,not available
4,1,281,Advanced,102,1,mvc2,cloud-services,0,not available


``Taking overall average of views of the users to compare their interests for various courses``

In [10]:
df_user_attr["mean_view_time"] = df_user_attr["view_time_seconds"].groupby(df_user_attr['user_handle_x']).transform('mean')
df_user_attr["mean_assessment_score"] = df_user_attr["user_assessment_score"].groupby(df_user_attr['user_handle_x']).transform('mean')
df_user_attr.head()

,user_handle_x,author_handle,level,view_time_seconds,user_handle_y,interest_tag,course_tags,user_assessment_score,assessment_tag,mean_view_time,mean_assessment_score
0,1,875,Beginner,3786,1,mvc2,sharepoint,0,not available,2035.75,0
1,1,875,Beginner,1098,1,mvc2,sharepoint,0,not available,2035.75,0
2,1,875,Intermediate,4406,1,mvc2,sharepoint,0,not available,2035.75,0
3,1,875,Intermediate,553,1,mvc2,sharepoint,0,not available,2035.75,0
4,1,281,Advanced,102,1,mvc2,cloud-services,0,not available,2035.75,0


In [11]:
df_final = df_user_attr[["user_handle_x","course_tags","level","interest_tag","assessment_tag","mean_view_time",
                         "mean_assessment_score"]]
df_final.columns.values[0] = 'user_handle'
df_final.head()

,user_handle,course_tags,level,interest_tag,assessment_tag,mean_view_time,mean_assessment_score
0,1,sharepoint,Beginner,mvc2,not available,2035.75,0
1,1,sharepoint,Beginner,mvc2,not available,2035.75,0
2,1,sharepoint,Intermediate,mvc2,not available,2035.75,0
3,1,sharepoint,Intermediate,mvc2,not available,2035.75,0
4,1,cloud-services,Advanced,mvc2,not available,2035.75,0


``Counting number of time each course was viewed along with their difficulty level. Also, counting assessment and interest tags if any``

In [12]:
df_transformed = pd.get_dummies(df_final, columns=["course_tags","level","interest_tag","assessment_tag"]).groupby(['user_handle','mean_view_time','mean_assessment_score'], as_index=False).sum()
df_transformed.head(7)

,user_handle,mean_view_time,mean_assessment_score,course_tags_.net,course_tags_.net-compiler-platform,course_tags_.net-core,course_tags_.net-debugging,course_tags_.net-distributed-systems,course_tags_2d-animation,course_tags_3d-animation,...,assessment_tag_revit-architecture-modeling-families,assessment_tag_security-for-hackers-and-developers,assessment_tag_sharepoint,assessment_tag_solidworks-core-skills,assessment_tag_the-scrum-framework,assessment_tag_unity-game-development-core-skills,assessment_tag_using-microsoft-office-2016,assessment_tag_video-production,assessment_tag_windows-server-management,assessment_tag_zbrush-core-skills
0,1,2035.750000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1491.862745,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,194.666667,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1763.846154,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1441.350000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,2259.500000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,8,2046.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


``Scaling the variables``

In [13]:
scaler = MinMaxScaler()
for i in df_transformed.columns.values.tolist()[1:]:
    df_transformed[i] = scaler.fit_transform(df_transformed[i])
df_transformed.head()

,user_handle,mean_view_time,mean_assessment_score,course_tags_.net,course_tags_.net-compiler-platform,course_tags_.net-core,course_tags_.net-debugging,course_tags_.net-distributed-systems,course_tags_2d-animation,course_tags_3d-animation,...,assessment_tag_revit-architecture-modeling-families,assessment_tag_security-for-hackers-and-developers,assessment_tag_sharepoint,assessment_tag_solidworks-core-skills,assessment_tag_the-scrum-framework,assessment_tag_unity-game-development-core-skills,assessment_tag_using-microsoft-office-2016,assessment_tag_video-production,assessment_tag_windows-server-management,assessment_tag_zbrush-core-skills
0,1,0.086393,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.063036,0.0,0.0,0.0,0.0,0.0,0.0,0.016667,0.010989,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.007329,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.074716,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.060867,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
print("Shape of final data frame: ", df_transformed.shape)

Shape of final data frame:  (8760, 1574)


``Calculating user similarity matrix based on the their activity``

In [21]:
def similarity_matrix(user):
    x = df_transformed.iloc[:,1:].values
    idx2user = dict(zip(df_transformed.index, df_transformed["user_handle"]))
    user2idx = {v:k for k,v in idx2user.items()}
    similarities = cosine_similarity(x)
    print(similarities.shape)
    similar_users_idx = similarities[user2idx.get(user)]
    print(similar_users_idx.shape)
    similar_users_idx = np.where(similar_users_idx > 0.5)[0].tolist()
    similar_users = [idx2user.get(i) for i in similar_users_idx]
    print("Users similar to user handle %s are" %(user), ", ".join(str(i) for i in similar_users))
    print("Cosine distance between the user handle %s and user handles " %(user) + ", ".join(str(i) for i in similar_users) + " is " +
          ", ".join(str("%.2f" %(i)) for i in similarities[user2idx.get(user)][similar_users_idx]) + " respectively.")
    return similar_users

In [25]:
similar_users = similarity_matrix(50)

(8760, 8760)
(8760,)
Users similar to user handle 50 are 50, 4339, 4948, 7256
Cosine distance between the user handle 50 and user handles 50, 4339, 4948, 7256 is 1.00, 0.56, 0.55, 0.63 respectively.


``Above, just for sanity check user similar to itself is itself which is confirmed. Secondly, the threshold for simialrity set here is about 45 degrees i.e. the user activity vectors at an angle of around 45 degree from the user in question are considered similar. Data frames below shows that which attributes make user 50 similar to user 4339.``

In [17]:
df_final[df_final["user_handle"] == 50]

,user_handle,course_tags,level,interest_tag,assessment_tag,mean_view_time,mean_assessment_score
1250,50,xamarin,Intermediate,azure-deployment,python,2634.807018,0
1251,50,xamarin,Intermediate,azure-deployment,python,2634.807018,0
1252,50,golang,Beginner,azure-deployment,python,2634.807018,0
1253,50,git,Beginner,azure-deployment,python,2634.807018,0
1254,50,xamarin,Intermediate,azure-deployment,python,2634.807018,0
1255,50,xamarin,Intermediate,azure-deployment,python,2634.807018,0
1256,50,mobile,Intermediate,azure-deployment,python,2634.807018,0
1257,50,mobile,Intermediate,azure-deployment,python,2634.807018,0
1258,50,mobile,Intermediate,azure-deployment,python,2634.807018,0
1259,50,mobile,Intermediate,azure-deployment,python,2634.807018,0


In [18]:
df_final[df_final["user_handle"]==4339]

,user_handle,course_tags,level,interest_tag,assessment_tag,mean_view_time,mean_assessment_score
107170,4339,mobile,Intermediate,it-fundamentals,not available,852.6,0
107171,4339,mobile,Intermediate,it-fundamentals,not available,852.6,0
107172,4339,android,Beginner,it-fundamentals,not available,852.6,0
107173,4339,xamarin,Intermediate,it-fundamentals,not available,852.6,0
107174,4339,mobile,Intermediate,it-fundamentals,not available,852.6,0


``Inserting user activity attributes into sqlite data base.``
 - Table Name: 'user_activity'
 - Database Name: 'users'

In [17]:
cnx = DatabaseWorker("users.db")
cnx.create_table("user_activity", df_final.columns.values.tolist())
cnx.insert_table("user_activity", df_final)

In [18]:
cnx.query_table('user_activity')

,user_handle,course_tags,level,interest_tag,assessment_tag,mean_view_time,mean_assessment_score
0,1,sharepoint,Beginner,mvc2,not available,2035.750000,0
1,1,sharepoint,Beginner,mvc2,not available,2035.750000,0
2,1,sharepoint,Intermediate,mvc2,not available,2035.750000,0
3,1,sharepoint,Intermediate,mvc2,not available,2035.750000,0
4,1,cloud-services,Advanced,mvc2,not available,2035.750000,0
5,1,cloud-computing,Intermediate,mvc2,not available,2035.750000,0
6,1,cloud-services,Beginner,mvc2,not available,2035.750000,0
7,1,javascript-libraries,Beginner,mvc2,not available,2035.750000,0
8,2,3d-lighting,Beginner,mvc-html-helpers,photoshop,1491.862745,0
9,2,interactive-design,Advanced,mvc-html-helpers,photoshop,1491.862745,0


```hgh```